In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
import os
from newspaper import Article
# pdfminer.six
# import pdfminer
# import pdfminer.high_level
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

In [2]:
headers = {
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }


def extract_urls(url):
    time.sleep(2)
    try:
        coverpage = requests.get(url, headers = headers).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        rows = soup.find_all('div', class_= 'o-layout-list__item l-m-100 l-t-50 l-d-33')
        rows = [str(i) for i in rows]
    except:
        rows = 'error'
    return rows

def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')

    link=  soup.a['href']
    if link.startswith('/'):
        link = 'https://www.rfi.fr' + link
    text = re.sub("^\s+|\s+$", "", soup.text, flags=re.UNICODE)
    text = re.sub("\n +(\n)*", "\n", text, flags=re.UNICODE).split('\n')
    if len(text) == 2:
        text = ['None'] + text
    tag, date, title = text[0],text[1],text[2]
     
    return pd.Series([tag, date, title, link])


def extract_text(url):
    time.sleep(1)
    article = Article(url, keep_article_html=False)
    article.download()
    article.parse()
    return article.text

In [3]:
tags = ['army','crime','police','protests','military',
        'war','war-crimes','civil-war',
        'violence','domestic-violence',
        'human-rights','refugees']

pages = []
for tag in tags:
    first_page = ['https://www.rfi.fr/en/tag/%s/'%tag]
    other_page = ['https://www.rfi.fr/en/tag/%s/%d/#pager' %(tag, i) for i in range(2,60)]   # The maxium pages are about 60.
    pages = pages + first_page + other_page


df = pd.DataFrame(pages,  columns =['url'] )
df

,url
0,https://www.rfi.fr/en/tag/army/
1,https://www.rfi.fr/en/tag/army/2/#pager
2,https://www.rfi.fr/en/tag/army/3/#pager
3,https://www.rfi.fr/en/tag/army/4/#pager
4,https://www.rfi.fr/en/tag/army/5/#pager
...,...
703,https://www.rfi.fr/en/tag/refugees/55/#pager
704,https://www.rfi.fr/en/tag/refugees/56/#pager
705,https://www.rfi.fr/en/tag/refugees/57/#pager
706,https://www.rfi.fr/en/tag/refugees/58/#pager


In [4]:
pandarallel.initialize(nb_workers= 128, progress_bar=True)
df[['rows']] = df['url'].parallel_apply(extract_urls)

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df

,url,rows
0,https://www.rfi.fr/en/tag/army/,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=..."
1,https://www.rfi.fr/en/tag/army/2/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20200422-cameroon-government-makes-u-turn-on-anglophone-massacre-in-ngarbuh"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""000_19Q8TJ"" aria-hidden=""true..."
2,https://www.rfi.fr/en/tag/army/3/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/europe/20180723-french-soldiers-battle-wildfires-sweden"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""A wildfire is seen in Karbole outside Ljusdal, Sweden, J..."
3,https://www.rfi.fr/en/tag/army/4/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20150627-Tunisia-confronted-dilemma-security-liberty-sousse-terror-attack-police-state"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Tunisian army sold..."
4,https://www.rfi.fr/en/tag/army/5/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20140522-african-press-review-22-may-2014"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt="""" aria-hidden=""true"" class=""m-figure__img"" data-image-dataset='..."
...,...,...
301,https://www.rfi.fr/en/tag/refugees/14/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20131018-french-press-review-18-october-2013"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt="""" aria-hidden=""true"" class=""m-figure__img"" data-image-datase..."
302,https://www.rfi.fr/en/tag/refugees/15/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20130402-extraordinary-eritreans-series-part-ii-tsedal-yohannes"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Tsedal Yohannes"" aria-hidden=""true"" class..."
303,https://www.rfi.fr/en/tag/refugees/16/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/asia-pacific/20120824-australia-boosts-its-refugee-intake"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Australia's Prime Minister Julia Gillard"" aria-hidden=..."
304,https://www.rfi.fr/en/tag/refugees/17/#pager,"[<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20120405-bangladesh-biharis-refugee-camps-v

In [15]:
df = df.explode('rows', ignore_index=True)
df

,url,rows
0,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""..."
1,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n..."
2,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""The submarine L'Inflexi..."
3,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""US counterterr..."
4,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""2020-09-25 China PLA Air Force Yun ..."
...,...,...
7240,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/asia-pacific/20100622-uzbeks-return-kyrgyzstan-after-ethnic-clashes"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Kyrgyz men carry the body of an unidentified ..."
7241,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20100621-unhcr-meet-senegal-refugees"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Senegal"" aria-hidden=""true"" class=""m-figure__img"" data-image-dataset=..."
7242,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20100618-kenyan-police-abusing-refugees-report-says"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Flag of Kenya"" aria-hidden=""true"" class=""m-figure__img..."
7243,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/middle-east/20100618-100000-iraqi-refugees-found-place-resettle"">\n<div class=""article

In [16]:
df.to_csv('rfi_raw.csv', index=False)

In [5]:
df = pd.read_csv('rfi_raw.csv', header=[0])
# df = df[~df.rows.isnull()].reset_index(drop=True)
df

,url,rows
0,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""..."
1,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n..."
2,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""The submarine L'Inflexi..."
3,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""US counterterr..."
4,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""2020-09-25 China PLA Air Force Yun ..."
...,...,...
7240,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/asia-pacific/20100622-uzbeks-return-kyrgyzstan-after-ethnic-clashes"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Kyrgyz men carry the body of an unidentified ..."
7241,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20100621-unhcr-meet-senegal-refugees"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Senegal"" aria-hidden=""true"" class=""m-figure__img"" data-image-dataset=..."
7242,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20100618-kenyan-police-abusing-refugees-report-says"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Flag of Kenya"" aria-hidden=""true"" class=""m-figure__img..."
7243,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/middle-east/20100618-100000-iraqi-refugees-found-place-resettle"">\n<div class=""article

In [12]:
df.url.str.split('/').apply(lambda x: x[5]).value_counts()

protests             1357
police               1049
violence              896
military              826
crime                 790
human-rights          658
war                   597
refugees              428
civil-war             239
army                  232
war-crimes            103
domestic-violence      70
Name: url, dtype: int64

In [69]:
pandarallel.initialize(progress_bar=True)
df[['tag','date','title', 'link']] = df['rows'].parallel_apply(extract_row)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [70]:
df

,url,rows,tag,date,title,link
0,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""...",French military,11/03/2021,"Is French Foreign Legion still an elite, international fighting force?",https://www.rfi.fr/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military
1,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n...",Spotlight on France,04/03/2021,"Podcast: Filming under a Covid cloud, surveying the French, the Foreign Legion",https://www.rfi.fr/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg
2,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""The submarine L'Inflexi...",Military,19/02/2021,French Navy to replace four nuclear submarines in multi-billion euro plan,https://www.rfi.fr/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan
3,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""US counterterr...",Hostage-taking,02/11/2020,US commandos rescue American hostage kidnapped in Niger,https://www.rfi.fr/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump
4,https://www.rfi.fr/en/tag/army/,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""2020-09-25 China PLA Air Force Yun ...",Taiwan under siege,25/09/2020,Is mainland China gearing up for an invasion of Taiwan?,https://www.rfi.fr/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan
...,...,...,...,...,...,...
7240,https://www.rfi.fr/en/tag/refugees/18/#pager,"<div class=""o-layout-list__item l-m-100 l-t-50 l-d-33"">\n<div class=""m-item-list-article"" data-item-article="""">\n<a data-item-article-link="""" href=""/en/asia-pacific/20100622-uzbeks-return-kyrgyzstan-after-ethnic-clashes"">\n<div class=""article__figure-wrapper"">\n<figure class=""m-figure m-figure--16x9"">\n<div class=""m-figure__img-wrapper"">\n<img alt=""Kyrgyz men carry the body of an unidentified ...",Kyrgyzstan - Uzbekistan,22/06/2010,Uzbeks return to Kyrgyzstan after ethnic clashes,htt

In [71]:
df = df[~df.duplicated(subset='link')].reset_index(drop=True)
df = df.iloc[:,2:]
df

,tag,date,title,link
0,French military,11/03/2021,"Is French Foreign Legion still an elite, international fighting force?",https://www.rfi.fr/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military
1,Spotlight on France,04/03/2021,"Podcast: Filming under a Covid cloud, surveying the French, the Foreign Legion",https://www.rfi.fr/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg
2,Military,19/02/2021,French Navy to replace four nuclear submarines in multi-billion euro plan,https://www.rfi.fr/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan
3,Hostage-taking,02/11/2020,US commandos rescue American hostage kidnapped in Niger,https://www.rfi.fr/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump
4,Taiwan under siege,25/09/2020,Is mainland China gearing up for an invasion of Taiwan?,https://www.rfi.fr/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan
...,...,...,...,...
5976,Culture in France,04/07/2010,Refugee All Stars - from Sierra Leone to New Orleans and back,https://www.rfi.fr/en/culture/20100604-refugee-all-stars-sierra-leone-new-orleans-and-back
5977,Angola,22/06/2010,Angolan refugees to be sent home,https://www.rfi.fr/en/africa/20100622-angolan-refugees-be-sent-home
5978,Kyrgyzstan - Uzbekistan,22/06/2010,Uzbeks return to Kyrgyzstan after ethnic clashes,https://www.rfi.fr/en/asia-pacific/20100622-uzbeks-return-kyrgyzstan-after-ethnic-clashes
5979,Senegal - from our correspondent,21/06/2010,UNHCR to meet Senegal refugees,https://www.rfi.fr/en/africa/20100621-unhcr-meet-senegal-refugees


In [78]:
# article = Article(url, keep_article_html=False)
# article.download()
# article.parse()
# print(article.text)

Soldiers of the Foreign Legion's pionniers regiment march during the annual Bastille Day parade July 14, 2019 on the Champs Elysées avenue in Paris.

France’s Foreign Legion, the national army's elite infantry force, has long attracted men the world over looking for adventure, a new start in life and the promise of French nationality. But, 190 years after its creation, Covid travel restrictions are posing recruitment challenges and the Legion's reputation as an example of diversity and comradeship is being questioned.

Advertising Read more

The closest most people in France will get to a légionnaire is on 14 July when a unit from the Foreign Legion marches down the Champs-Elysées in Paris as part of the Bastille Day military parade.

The Legion’s pionniers regiment, with their long beards, white caps and leather aprons, make for memorable photos. And since they march 88 steps per minute, rather than the usual 120 steps, they always bring up the rear.

The legionnaires march to a diffe

In [3]:
df = pd.read_csv('rfi_final.csv', header=[0])
df

,tag,date,title,link,text
0,French military,11/03/2021,"Is French Foreign Legion still an elite, international fighting force?",https://www.rfi.fr/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military,"The closest most people in France will get to a légionnaire is on 14 July when a unit from the Foreign Legion marches down the Champs-Elysées in Paris as part of the Bastille Day military parade.\nThe Legion’s pionniers regiment, with their long beards, white caps and leather aprons, make for memorable photos. And since they march 88 steps per minute, rather than the usual 120 steps, they alwa..."
1,Spotlight on France,04/03/2021,"Podcast: Filming under a Covid cloud, surveying the French, the Foreign Legion",https://www.rfi.fr/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg,"The French movie industry is booming, despite Covid lockdowns and the risk of transmitting the virus on set. The first Covid lockdown in March 2020 shut down productions in France, but since then filming has picked up speed. Some 450 productions have been shot in France since May 2020 (compared to 300 in 2019). But Covid has meant people work differently, and productions are more expensive. We..."
2,Military,19/02/2021,French Navy to replace four nuclear submarines in multi-billion euro plan,https://www.rfi.fr/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan,"Fifty years ago France employed its first ever nuclear submarine, the Redoutable. In 1997, the Redoutable and its five sister ships were replaced by the four Triomphant class SSBNs (""Submersible Ship carrying Ballistic Nuclear missiles,) the Triomphant (deployed in 1997), the Téméraire (1999), the Vigilant (2004) and the Terrible (2010).\nThe ships are typically 138 meters long and 12.5 metres..."
3,Hostage-taking,02/11/2020,US commandos rescue American hostage kidnapped in Niger,https://www.rfi.fr/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump,"Walton was abducted last Tuesday morning on the outskirts of Massalata, a village about 10 kilometres from the border with Nigeria.\nIn a dramatic predawn raid on Saturday, commandos from the U.S. Navy’s elite SEAL Team 6 rescued the 27-year-old after they tracked his abductors' phones to a hide-out in neighbouring northern Nigeria, according to U.S. officials.\nWalton was released unharmed an..."
4,Taiwan under siege,25/09/2020,Is mainland China gearing up for an invasion of Taiwan?,https://www.rfi.fr/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan,"Chinese military exercises ""are rehearsals for taking over Taiwan,” said the Beijing-controlled Global Times newspaper in an editorial published on 18 September. “What is needed is a political reason that can turn them into a real battle to smash Taiwan independence forces.”\nAccording to Taiwan News, the exercises by the mainland’s People’s Liberation Army involved dozens of transgressions in..."
...,...,...,...,...,...
5976,Culture in France,04/07/2010,Refugee All Stars - from Sierra Leone to New Orleans and back,https://www.rfi.fr/en/culture/20100604-refugee-all-stars-sierra-leone-new-orleans-and-back,error p
5977,Angola,22/06/2010,Angolan refugees to be sent home,https://www.rfi.fr/en/africa/20100622-angolan-refugees-be-sent-home,error p
5978,Kyrgyzstan - Uzbekistan,22/06/2010,Uzbeks return to Kyrgyzstan after ethnic clashes,https://www.rfi.fr/en/asia-pacific/20100622-uzbeks-return-kyrgyzstan-after-ethnic-clashes,error p
5979,Senegal - from our correspondent,21/06/2010,UNHCR to meet Senegal refugees,https://www.rfi.fr/en/africa/20100621-unhcr-meet-senegal-refugees,error p


In [4]:
def extract_text(url):
    try:
#         time.sleep(1)
        coverpage = requests.get(url, headers = headers).content
    except:
        print('error url', url)
        return 'error url'
    
    soup = BeautifulSoup(coverpage, 'html.parser')
    bold = soup.find_all('p', class_= 't-content__chapo')
    bold = bold[0].text if len(bold) else ''

    body = soup.find_all('div', class_= 't-content__body u-clearfix')
    if len(body):
        paragraph = body[0].find_all('p')
        paragraph = [p.text for p in paragraph]
        paragraph = '\n'.join(paragraph)
    else:
        paragraph = ''
    text = bold + paragraph
    
    if len(text)==0:
        print('error text', url)
        text = 'error text'   
    return text

In [5]:
pandarallel.initialize(nb_workers=8, progress_bar=False)
batch_size = 256

start = time.time()
for i in range(0,df.shape[0],batch_size):
    df.loc[i:i+batch_size-1, 'text']=\
    df.loc[i:i+batch_size-1, 'link'].parallel_apply(extract_text).values
    
    end = time.time()
    print('--- %4d --- %.1f'%(i, end-start))
    start = end

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
---    0 --- 15.8
---  256 --- 4.9
---  512 --- 4.1
---  768 --- 55.6
--- 1024 --- 57.2
--- 1280 --- 69.7
--- 1536 --- 99.4
--- 1792 --- 118.1
--- 2048 --- 97.7
--- 2304 --- 117.1
--- 2560 --- 128.4
--- 2816 --- 93.3
--- 3072 --- 125.8
--- 3328 --- 119.0
--- 3584 --- 126.1
--- 3840 --- 128.9
--- 4096 --- 123.9
error text https://www.rfi.fr/en/middle-east/20150320-Syrian-refugees-in-Lebanon-series-part-5/5
--- 4352 --- 121.1
--- 4608 --- 134.2
--- 4864 --- 99.2
--- 5120 --- 85.5
--- 5376 --- 135.5
--- 5632 --- 190.7
--- 5888 --- 51.8


In [10]:
df = df[df.text != 'error text'].reset_index(drop= True)
df

,tag,date,title,link,text
0,French military,11/03/2021,"Is French Foreign Legion still an elite, international fighting force?",https://www.rfi.fr/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military,"\n France’s Foreign Legion, the national army's elite infantry force, has long attracted men the world over looking for adventure, a new start in life and the promise of French nationality. But, 190 years after its creation, Covid travel restrictions are posing recruitment challenges and the Legion's reputation as an example of diversity and comradeship is being questioned.\..."
1,Spotlight on France,04/03/2021,"Podcast: Filming under a Covid cloud, surveying the French, the Foreign Legion",https://www.rfi.fr/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg,"\n Why is France's Covid vaccination drive lagging behind other countries? Also, French film and TV productions are booming, despite new restrictions on set to stop the virus spreading. A large-scale health survey of tens of thousands of French people yields troves of information. And Foreign Legion recruitment woes as the elite unit marks its 190th anniversary. \n ..."
2,Military,19/02/2021,French Navy to replace four nuclear submarines in multi-billion euro plan,https://www.rfi.fr/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan,"\n France's Navy is to purchase four nuclear submarines worth several billion euros, French Minister of Defence Florence Parly has announced. The vessels are scheduled to be operational by 2035 and to be part of a military operation that appears increasingly assertive.\n Fifty years ago France employed its first ever nuclear submarine, the Redoutable. In 1997,..."
3,Hostage-taking,02/11/2020,US commandos rescue American hostage kidnapped in Niger,https://www.rfi.fr/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump,"\n US commandos have rescued missionary Philip Walton in Nigeria, just a few days after he was abducted by gunmen in the south of neighbouring Niger and taken across the border.\n Walton was abducted last Tuesday morning on the outskirts of Massalata, a village about 10 kilometres from the border with Nigeria.\nIn a dramatic predawn raid on Saturday, commandos..."
4,Taiwan under siege,25/09/2020,Is mainland China gearing up for an invasion of Taiwan?,https://www.rfi.fr/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan,"\n Increasingly hostile rhetoric from Beijing, and a growing number of incursions by the Chinese air force and navy into Taiwanese territory, point to China’s determination to occupy the self-proclaimed island state. Will the US honour promises to Taipei and come to the rescue if push comes to shove?\n Chinese military exercises ""are rehearsals for taking over..."
...,...,...,...,...,...
5975,Culture in France,04/07/2010,Refugee All Stars - from Sierra Leone to New Orleans and back,https://www.rfi.fr/en/culture/20100604-refugee-all-stars-sierra-leone-new-orleans-and-back,"\n Sierra Leone’s Refugee All Stars are back with a second album, Rise & Shine. It’s a collection of traditional west African music and reggae, recorded together with New Orleans musicians. The band extends its interests beyond music with a new project, “We Own TV”, training young Sierra Leoneans in documentary making.\n Four years after the successful documen..."
5976,Angola,22/06/2010,Angolan refugees to be sent home,https://www.rfi.fr/en/africa/20100622-angolan-refugees-be-sent-home,"\n About 100,000 Angolan refugees will be urged to return home next year, the United Nations refugee agency said Tuesday. At the end of 2011, Angolans living in neighbouring countries will no longer be officially considered as refugees, meaning they will either have to seek

In [11]:
df.to_csv('rfi_final.csv', index=False)

In [ ]:
quit()

In [2]:
df = pd.read_csv('rfi_final.csv', header = [0])
df

,tag,date,title,link,text
0,French military,11/03/2021,"Is French Foreign Legion still an elite, international fighting force?",https://www.rfi.fr/en/france/20210311-is-french-foreign-legion-still-an-elite-international-fighting-force-covid-19-diversity-military,"\n France’s Foreign Legion, the national army's elite infantry force, has long attracted men the world over looking for adventure, a new start in life and the promise of French nationality. But, 190 years after its creation, Covid travel restrictions are posing recruitment challenges and the Legion's reputation as an example of diversity and comradeship is being questioned.\..."
1,Spotlight on France,04/03/2021,"Podcast: Filming under a Covid cloud, surveying the French, the Foreign Legion",https://www.rfi.fr/en/podcasts/spotlight-on-france/20210304-podcast-filming-under-a-covid-cloud-surveying-the-french-the-foreign-legion-légion-etrangère-cinema-constances-gainsbourg,"\n Why is France's Covid vaccination drive lagging behind other countries? Also, French film and TV productions are booming, despite new restrictions on set to stop the virus spreading. A large-scale health survey of tens of thousands of French people yields troves of information. And Foreign Legion recruitment woes as the elite unit marks its 190th anniversary. \n ..."
2,Military,19/02/2021,French Navy to replace four nuclear submarines in multi-billion euro plan,https://www.rfi.fr/en/france/20210219-french-navy-to-replace-four-nuclear-submarines-in-multi-billion-euro-plan,"\n France's Navy is to purchase four nuclear submarines worth several billion euros, French Minister of Defence Florence Parly has announced. The vessels are scheduled to be operational by 2035 and to be part of a military operation that appears increasingly assertive.\n Fifty years ago France employed its first ever nuclear submarine, the Redoutable. In 1997,..."
3,Hostage-taking,02/11/2020,US commandos rescue American hostage kidnapped in Niger,https://www.rfi.fr/en/africa/20201102-u-s-commandos-rescue-american-hostage-philip-walton-kidnapped-in-niger-niger-trump,"\n US commandos have rescued missionary Philip Walton in Nigeria, just a few days after he was abducted by gunmen in the south of neighbouring Niger and taken across the border.\n Walton was abducted last Tuesday morning on the outskirts of Massalata, a village about 10 kilometres from the border with Nigeria.\nIn a dramatic predawn raid on Saturday, commandos..."
4,Taiwan under siege,25/09/2020,Is mainland China gearing up for an invasion of Taiwan?,https://www.rfi.fr/en/international/20200925-is-mainland-china-gearing-up-for-an-invasion-of-taiwan,"\n Increasingly hostile rhetoric from Beijing, and a growing number of incursions by the Chinese air force and navy into Taiwanese territory, point to China’s determination to occupy the self-proclaimed island state. Will the US honour promises to Taipei and come to the rescue if push comes to shove?\n Chinese military exercises ""are rehearsals for taking over..."
...,...,...,...,...,...
5975,Culture in France,04/07/2010,Refugee All Stars - from Sierra Leone to New Orleans and back,https://www.rfi.fr/en/culture/20100604-refugee-all-stars-sierra-leone-new-orleans-and-back,"\n Sierra Leone’s Refugee All Stars are back with a second album, Rise & Shine. It’s a collection of traditional west African music and reggae, recorded together with New Orleans musicians. The band extends its interests beyond music with a new project, “We Own TV”, training young Sierra Leoneans in documentary making.\n Four years after the successful documen..."
5976,Angola,22/06/2010,Angolan refugees to be sent home,https://www.rfi.fr/en/africa/20100622-angolan-refugees-be-sent-home,"\n About 100,000 Angolan refugees will be urged to return home next year, the United Nations refugee agency said Tuesday. At the end of 2011, Angolans living in neighbouring countries will no longer be officially considered as refugees, meaning they will either have to seek

In [4]:
len(df.tag.unique())

1674